In [1]:
from util import load_user_data
import pandas as pd
import numpy as np

valid_users, tp_data = load_user_data(folder_name="../anonymized_data", reserved_users=None)

248 complete the NASA-TLX
248 complete the whole study


In [2]:
user2condition = tp_data['user2condition']
condition_count = {}
for user in valid_users:
    tp_condition = user2condition[user]
    if tp_condition not in condition_count:
        condition_count[tp_condition] = 0
    condition_count[tp_condition] += 1
print(condition_count)

{'AP-UE': 64, 'AP-AE': 63, 'UP-AE': 61, 'UP-UE': 60}


In [3]:
variable_dict = {}
trust_dimensions = ["Reliability/Competence", "Understanding/Predictability", 
                    "Intention of Developers", "Trust in Automation"]
covariates_1 = ["Propensity to Trust", "Familiarity"]
covariates = ["llm_expertise", "assistant_expertise"]
all_conditions = ["AP-AE", "AP-UE", "UP-AE", "UP-UE"]
condition_dict = {}
for condition in all_conditions:
    condition_dict[condition] = {}
    for dimension in trust_dimensions:
        condition_dict[condition][dimension] = []
    condition_dict[condition]["calibrated_trust_planning"] = []
    condition_dict[condition]["calibrated_trust_execution"] = []
    condition_dict[condition]["RAIR"] = []
    condition_dict[condition]["RSR"] = []

for dimension in trust_dimensions:
    variable_dict[dimension] = []
for dimension in covariates_1:
    variable_dict[dimension] = []
for dimension in covariates:
    variable_dict[dimension] = []

user_expertise = tp_data["user_expertise"]
trust = tp_data["trust_in_automation"]
calibrated_trust_planning = tp_data["calibrated_trust_planning"]
calibrated_trust_execution = tp_data["calibrated_trust_execution"]
variable_dict["condition"] = []
variable_dict["planning"] = []
variable_dict["execution"] = []
variable_dict["calibrated_trust_planning"] = []
variable_dict["calibrated_trust_execution"] = []
for user in valid_users:
    tp_condition = user2condition[user]
    if tp_condition not in all_conditions:
        # ignore pilot study
        continue
    variable_dict["condition"].append(tp_condition)
    if tp_condition.startswith("AP"):
        variable_dict["planning"].append("automatic")
    else:
        variable_dict["planning"].append("user-involved")
    if tp_condition.endswith("AE"):
        variable_dict["execution"].append("automatic")
    else:
        variable_dict["execution"].append("user-involved")
    variable_dict["calibrated_trust_planning"].append(calibrated_trust_planning[user]["avg"])
    condition_dict[tp_condition]["calibrated_trust_planning"].append(calibrated_trust_planning[user]["avg"])
    variable_dict["calibrated_trust_execution"].append(calibrated_trust_execution[user]["avg"])
    condition_dict[tp_condition]["calibrated_trust_execution"].append(calibrated_trust_execution[user]["avg"])
    for dimension in trust_dimensions:
        variable_dict[dimension].append(trust[user][dimension])
        condition_dict[tp_condition][dimension].append(trust[user][dimension])
    for dimension in ["Propensity to Trust", "Familiarity"]:
        variable_dict[dimension].append(trust[user][dimension])
    for dimension in covariates:
        variable_dict[dimension].append(user_expertise[user][dimension])
df = pd.DataFrame(variable_dict)

In [4]:
from pingouin import ancova, anova
from statsmodels.stats.multicomp import pairwise_tukeyhsd
dimension = "calibrated_trust_planning"
# aov = anova(dv=dimension, between=['planning', 'execution'], data=df, effsize='n2')
aov = anova(dv=dimension, between=['planning'], data=df, effsize='n2')
# aov = ancova(dv=dimension, covar=["Propensity to Trust", "Familiarity", "llm_expertise", "assistant_expertise"], between='planning', data=df, effsize='n2')
print(aov.round(3))
if aov.to_dict()['p-unc'][0] <= 0.05 / 4:
    tukey = pairwise_tukeyhsd(endog=df[dimension], groups=df['planning'], alpha=0.0125)
    print(tukey)
for condition in all_conditions:
    print(condition, np.mean(condition_dict[condition][dimension]))
print("-" * 17)

     Source  ddof1  ddof2      F  p-unc   n2
0  planning      1    246  0.063  0.802  0.0
AP-AE 0.5052910052910052
AP-UE 0.49479166666666663
UP-AE 0.4972677595628415
UP-UE 0.4944444444444444
-----------------


/Users/ghe/opt/anaconda3/lib/python3.9/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.3, the latest is 0.5.5.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [11]:
from pingouin import ancova, anova
from statsmodels.stats.multicomp import pairwise_tukeyhsd
dimension = "calibrated_trust_execution"
aov = anova(dv=dimension, between=['planning', 'execution'], data=df, effsize='n2')
# aov = anova(dv=dimension, between=['execution'], data=df, effsize='n2')
# aov = ancova(dv=dimension, covar=["Propensity to Trust", "Familiarity", "llm_expertise", "assistant_expertise"], between='execution', data=df, effsize='n2')
print(aov.round(3))
if aov.to_dict()['p-unc'][0] <= 0.05 / 4:
    tukey = pairwise_tukeyhsd(endog=df[dimension], groups=df['condition'], alpha=0.0125)
    print(tukey)
for condition in all_conditions:
    print(condition, np.mean(condition_dict[condition][dimension]))
print("-" * 17)

                 Source     SS     DF     MS      F  p-unc     n2
0              planning  0.054    1.0  0.054  1.423  0.234  0.006
1             execution  0.001    1.0  0.001  0.023  0.878  0.000
2  planning * execution  0.013    1.0  0.013  0.340  0.560  0.001
3              Residual  9.271  244.0  0.038    NaN    NaN    NaN
AP-AE 0.6587301587301588
AP-UE 0.6484375
UP-AE 0.6147540983606558
UP-UE 0.6333333333333333
-----------------
